In [2]:
import pyspark
import os
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import jaccard_score

sc = pyspark.SparkContext(appName="BigDataProject")

sc

<SparkContext master=local[*] appName=BigDataProject>

In [3]:
#sc.stop()

In [4]:
import variables
#List of english function words
function_words = variables.function_words

#List of punctuations char
punctuations_char = variables.punctuations_char

#Dataset path
my_dir = "./BooksDataset"

# False if you don't have the rdd saved, True otherwise
readFromSavedFile = True

#List of Authors
authors = variables.authors

#Number of Authors
n_authors = len(authors)

In [5]:
#Save/read all books' name of every author

if not readFromSavedFile:
    for i in range(n_authors):
        book = []
        mydiraut = my_dir+"/"+authors[i]
        print(f"Getting books' name for author: {authors[i]}")
        
        for file in os.listdir(mydiraut):
            d = os.path.join(mydiraut, file)
            book.append(file)
            
        print(f"Start - Saving RDD of all books' name of author: {authors[i]}")
        sc.parallelize(book).saveAsTextFile("./RDD/BooksName/"+authors[i])
        print(f"End - Saving RDD of all books' name of author: {authors[i]}\n")
    

booksName = []

for i in range(n_authors):
    print(f" Start - Reading all books' name from saved RDD of author: {authors[i]}")
    booksName.append(sc.textFile("./RDD/BooksName/" + authors[i] + "/*").collect()) 
    print(f" End - Reading all books' name from saved RDD of author: {authors[i]}")
    


 Start - Reading all books' name from saved RDD of author: Arthur Conan Doyle
 End - Reading all books' name from saved RDD of author: Arthur Conan Doyle
 Start - Reading all books' name from saved RDD of author: Charles Dickens
 End - Reading all books' name from saved RDD of author: Charles Dickens
 Start - Reading all books' name from saved RDD of author: Daniel Defoe
 End - Reading all books' name from saved RDD of author: Daniel Defoe
 Start - Reading all books' name from saved RDD of author: Edith Wharton
 End - Reading all books' name from saved RDD of author: Edith Wharton
 Start - Reading all books' name from saved RDD of author: Jane Austen
 End - Reading all books' name from saved RDD of author: Jane Austen
 Start - Reading all books' name from saved RDD of author: Joseph Conrad
 End - Reading all books' name from saved RDD of author: Joseph Conrad
 Start - Reading all books' name from saved RDD of author: Lewis Carroll
 End - Reading all books' name from saved RDD of author

In [6]:
#Function to clear the text given in input

def clearText(file_path):
    return sc.textFile(file_path).filter(bool)  \
                .map(lambda w: w.replace("."," . ")) \
                .map(lambda w: w.replace(","," , ")) \
                .map(lambda w: w.replace(";"," ; ")) \
                .map(lambda w: w.replace(":"," : ")) \
                .map(lambda w: w.replace("!"," ! ")) \
                .map(lambda w: w.replace("?"," ? ")) \
                .map(lambda w: w.replace('"',' " ')) \
                .map(lambda w: w.replace("”"," ” ")) \
                .map(lambda w: w.replace("“"," “ ")) \
                .map(lambda w: w.replace("’"," ’ ")) \
                .map(lambda w: w.replace("‘"," ‘ ")) \
                .map(lambda w: w.replace("-"," - ")) \
                .map(lambda w: w.replace("_"," _ ")) \
                .map(lambda w: w.replace("{"," { ")) \
                .map(lambda w: w.replace("}"," } ")) \
                .map(lambda w: w.replace("["," [ ")) \
                .map(lambda w: w.replace("]"," ] ")) \
                .map(lambda w: w.replace("("," ( ")) \
                .map(lambda w: w.replace(")"," ) ")) \
                .map(lambda w: w.replace("*"," * "))


In [7]:
#WordCount returning a list of words
#.map(lambda w: (w[0][1:len(w[0])], w[1]) if (w[0][0] == "'") else w) \ dopo reduceByKey

def wordCount(file_path):
    words = clearText(file_path) \
                .flatMap(lambda line: line.split(" ")) \
                .map(lambda w: w.lower()) \
                .map(lambda w: (w, 1)) \
                .reduceByKey(lambda v1, v2: v1 + v2) \
                .filter(lambda w: w[0] != "") \
                .sortBy(lambda w: -w[1])
    
    result = words.collect()

    # Calculate total words of the book given in input
    total_words = 0
    for t in result:
        if t[0] not in punctuations_char:
            total_words += t[1]
            
    # Add the total word to the list of words
    result.append(('total_words', total_words))  
    
    return sc.parallelize(result)

In [8]:
#Saving word count RDD for every book for all authors

if not readFromSavedFile:

    for i in range(n_authors): #n_authors
        # Obtain the path of author directory
        tmp_filename = os.path.join(my_dir, authors[i])
        print(f"Word count of {authors[i]}'s books.")
        
        for j in range(len(booksName[i])): #len(booksName[i])
            # Obtain the path of books given the path of an author
            input_filename = os.path.join(tmp_filename, booksName[i][j])
            
            print(f"Start - Word count of book: {booksName[i][j]}")
            wordsOfABook = wordCount(input_filename)
            print(f"End - Word count of book: {booksName[i][j]}\n")
            
            print(f"Start - Saving word count of book: {booksName[i][j]}")
            wordsOfABook.saveAsTextFile("./RDD/Books/SingleBooks/"+ authors[i] + "/" + booksName[i][j])
            print(f"End - Saving word count of book: {booksName[i][j]}\n")
            
        

In [9]:
#Given a list of string convert it to a list of tuple
def stringToTuple(wordCount):

    result = []
    for s in wordCount:
        #Remove useless characters in the string
        res = s.strip("(").strip(")").split(", ")   
        tmp = len(res[0])
        #Take only the string inside the ''
        result.append((str(res[0][1:tmp-1]), float(res[1])))
        
    return result

In [10]:
#Read saved word count RDD for every book for all authors
wordsCountAuthorsBooks = [[] for _ in range(n_authors)]

for i in range(n_authors):
    print(f"Reading books for: {authors[i]}")
    for j in range(len(booksName[i])):
        # Getting path of books' name RDD
        tmp = "./RDD/Books/SingleBooks/" + authors[i] + "/" + booksName[i][j] + "/*"

        print(f"Start - Reading words count of book: {booksName[i][j]}")
        result = sc.textFile(tmp).collect()
        # Converting list of string into list of tuple
        result = stringToTuple(result)
        print(f"End - Reading words count of book: {booksName[i][j]}\n")

        wordsCountAuthorsBooks[i].append(result)
        

Reading books for: Arthur Conan Doyle
Start - Reading words count of book: Arthur Conan Doyle_Beyond the City.txt
End - Reading words count of book: Arthur Conan Doyle_Beyond the City.txt

Start - Reading words count of book: Arthur Conan Doyle_The Adventure of the Cardboard Box.txt


End - Reading words count of book: Arthur Conan Doyle_The Adventure of the Cardboard Box.txt

Start - Reading words count of book: Arthur Conan Doyle_The Adventure of the Dying Detective.txt
End - Reading words count of book: Arthur Conan Doyle_The Adventure of the Dying Detective.txt

Start - Reading words count of book: Arthur Conan Doyle_The Adventure of the Red Circle.txt
End - Reading words count of book: Arthur Conan Doyle_The Adventure of the Red Circle.txt

Start - Reading words count of book: Arthur Conan Doyle_The Adventures of Gerard.txt
End - Reading words count of book: Arthur Conan Doyle_The Adventures of Gerard.txt

Start - Reading words count of book: Arthur Conan Doyle_The Adventures of Sherlock Holmes.txt
End - Reading words count of book: Arthur Conan Doyle_The Adventures of Sherlock Holmes.txt

Start - Reading words count of book: Arthur Conan Doyle_The British Campaign in France and Flanders 1914.txt
End - Reading words count of book: Arthur Conan Doyle_The British

# Methods NO ML

In [11]:
#Calculating Word Count of books in training set for every authors  
training_index = [0, 1, 2, 3, 4, 5, 6]

if not readFromSavedFile:
    for a in range(n_authors):

        print(f"Author: {authors[a]}")
        empty_rdd = sc.emptyRDD()
        #Take only the books in training_index
        for b in training_index: #training_index:
            empty_rdd = empty_rdd.union(sc.parallelize(wordsCountAuthorsBooks[a][b]))
        
        #Word Count of all books in training_index of the author
        print("Start - Word Count of all books in training_index")
        # Filter to remove function words and punctuations char
        result = empty_rdd.reduceByKey(lambda v1, v2: v1 + v2) \
                .filter(lambda w: w[0] not in function_words) \
                .filter(lambda w: w[0] not in punctuations_char) \
                .sortBy(lambda w: -w[1])             
        print("End - Word Count of all books in training_index")
        
        #Take total words of the training set
        total_words = result.take(1)[0][1]
        print(f"Total words: {total_words}")
        
        # Normalization
        print("Start - Normalization of word's frequency")
        result = result.map(lambda x: (x[0], x[1]/total_words))
        print("End - Normalization of word's frequency")
        
        # Only save top 200 words 
        result = sc.parallelize(result.take(200))
        print("Start - Saving RDD Training set")
        result.saveAsTextFile(f"./RDD/Books/TrainingResult/{authors[a]}")
        print("End - Saving RDD Training set\n")    

    
trainingSetWordCountAuthors = []

#Reading RDD of authors training set result
for i in range(n_authors):
    print(f"Author: {authors[i]}")
 
    tmp = f"./RDD/Books/TrainingResult/{authors[i]}/*" 

    print(f"Start - Reading RDD Training set")
    result = sc.textFile(tmp).collect()

    # Converting list of string into list of tuple
    result = stringToTuple(result)
    print(f"End - Reading RDD Training set\n")

    trainingSetWordCountAuthors.append(result)



Author: Arthur Conan Doyle
Start - Reading RDD Training set
End - Reading RDD Training set

Author: Charles Dickens
Start - Reading RDD Training set
End - Reading RDD Training set

Author: Daniel Defoe
Start - Reading RDD Training set
End - Reading RDD Training set

Author: Edith Wharton
Start - Reading RDD Training set
End - Reading RDD Training set

Author: Jane Austen
Start - Reading RDD Training set
End - Reading RDD Training set

Author: Joseph Conrad
Start - Reading RDD Training set
End - Reading RDD Training set

Author: Lewis Carroll
Start - Reading RDD Training set
End - Reading RDD Training set

Author: Louisa May Alcott
Start - Reading RDD Training set
End - Reading RDD Training set

Author: Voltaire
Start - Reading RDD Training set
End - Reading RDD Training set

Author: William Shakespeare
Start - Reading RDD Training set
End - Reading RDD Training set



In [12]:
def testing(method):
    
    # books of author to test
    testing_index = [7, 8, 9]
    # result of test
    testPredictions = [[] for _ in range(n_authors)] 
    
    # give the predictions for every book in testing index of all authors
    for a in range(n_authors):
        for b in testing_index:
            # word count of the book to test
            bookToTest = wordsCountAuthorsBooks[a][b]
            # take total words of the book
            total_words = bookToTest[len(bookToTest)-1][1] 
            #take only top 100 words
            top100 = []
            count = 0
            # Find the top100 words that are not in 'function_words' and not in 'punctuations_char' + normalization
            for w in bookToTest:
                if (w[0] not in function_words) and (w[0] not in punctuations_char):
                    # Normalization
                    temp = (w[0], w[1]/total_words)
                    top100.append(temp)
                    count += 1
                if count == 100:
                    break
                    
            # predict the author of the book and save the prediction
            if method == 1:
                testPredictions[a].append(method1_words_in_common(top100)) 
            elif method == 2:
                testPredictions[a].append(method2_cosine_similarity(top100))
            elif method == 3:
                testPredictions[a].append(method3_jaccard_similarity(top100))
            
    modelAccuracy = 0
    # Calculate the accuracy of the model
    for a in range(n_authors):
        print(f"\nAuthor: {authors[a]}")
        predictions = testPredictions[a]
        print(predictions)
        
        accuracy = 0
        for p in predictions:
            if method == 1:
                if a in p:
                    accuracy += 1
            else:
                if a == p:
                    accuracy += 1
                
        modelAccuracy += accuracy
        accuracy = accuracy/len(predictions)*100
        print(f"Accuracy to predict the author= {accuracy}%")
        
    # Print the model accuracy
    tot = sum(len(sublist) for sublist in testPredictions)    
    modelAccuracy = modelAccuracy/tot*100
    print(f"\nModel Accuracy= {modelAccuracy}%")

# Method 1: Only based on words in common

In [13]:
#Function to count words in common given two lists of tuple. Tuple=(int, 'word')
def count_common_words(list1, list2):
    count_words = 0
    
    list2top100 = list2[1:101]
    
    #count words in common of the 2 lists given in input
    for w in list1:
        for w2 in list2top100:
            if w[0] == w2[0]:
                count_words += 1
                continue
                    
    return count_words


In [14]:
# Predict the author of the book given in input
def method1_words_in_common(top100):
    max_count = -1
    author_index = []
    
    #Check which authors have the greatest number of words in common compared to the book in input
    for a in range(n_authors):
        count = count_common_words(top100,trainingSetWordCountAuthors[a])
        if count == max_count:
            author_index.append(a)
    
        if count > max_count:
            author_index = [a]
            max_count = count 
    
    return author_index

In [15]:
method = 1
testing(method)


Author: Arthur Conan Doyle
[[5], [5], [1]]
Accuracy to predict the author= 0.0%

Author: Charles Dickens
[[1], [1], [1]]
Accuracy to predict the author= 100.0%

Author: Daniel Defoe
[[2], [2], [2]]
Accuracy to predict the author= 100.0%

Author: Edith Wharton
[[3], [0], [3]]
Accuracy to predict the author= 66.66666666666666%

Author: Jane Austen
[[4], [4], [4]]
Accuracy to predict the author= 100.0%

Author: Joseph Conrad
[[5], [5], [5]]
Accuracy to predict the author= 100.0%

Author: Lewis Carroll
[[6], [7], [6]]
Accuracy to predict the author= 66.66666666666666%

Author: Louisa May Alcott
[[7], [7], [7]]
Accuracy to predict the author= 100.0%

Author: Voltaire
[[7, 8], [8], [8]]
Accuracy to predict the author= 100.0%

Author: William Shakespeare
[[9], [9], [8]]
Accuracy to predict the author= 66.66666666666666%

Model Accuracy= 80.0%


# Method 2: Using cosine similarily

In [16]:
# Predict the author of the book given in input
def method2_cosine_similarity(list1top100):
    
    list1top100Words = [tup[0] for tup in list1top100]
    max_coseno = -1
    author_index = -1
    
    #Check which authors have the greatest number of words in common compared to the book in input
    for a in range(n_authors):
        wordsForCosineSimilarity = []
        for w in list1top100Words:
            wordsForCosineSimilarity.append(w)
        
        #Take top 100 of training set wordcount
        list2top100 = trainingSetWordCountAuthors[a][1:101]
        list2top100Words = [tup[0] for tup in list2top100]
        for w in list2top100Words:
            if w not in wordsForCosineSimilarity:
                wordsForCosineSimilarity.append(w)
        
        wordsValuesList1 = [0 for _ in range(len(wordsForCosineSimilarity))]
        for i, w in enumerate(wordsForCosineSimilarity):
            if w in list1top100Words:
                wordsValuesList1[i] = list1top100[list1top100Words.index(w)][1]
                
        wordsValuesList2 = [0 for _ in range(len(wordsForCosineSimilarity))]
        for i, w in enumerate(wordsForCosineSimilarity):
            if w in list2top100Words:
                wordsValuesList2[i] = list2top100[list2top100Words.index(w)][1]
        
        array_frequenze1 = np.array(wordsValuesList1).reshape(1, -1)
        array_frequenze2 = np.array(wordsValuesList2).reshape(1, -1)
        coseno = cosine_similarity(array_frequenze1, array_frequenze2)
    
        if coseno > max_coseno:
            author_index = a
            max_coseno = coseno 
            
    return author_index

In [17]:
method = 2
testing(method)


Author: Arthur Conan Doyle
[5, 5, 5]
Accuracy to predict the author= 0.0%

Author: Charles Dickens
[7, 1, 1]
Accuracy to predict the author= 66.66666666666666%

Author: Daniel Defoe
[2, 2, 8]
Accuracy to predict the author= 66.66666666666666%

Author: Edith Wharton
[3, 3, 3]
Accuracy to predict the author= 100.0%

Author: Jane Austen
[4, 4, 4]
Accuracy to predict the author= 100.0%

Author: Joseph Conrad
[5, 5, 5]
Accuracy to predict the author= 100.0%

Author: Lewis Carroll
[6, 5, 6]
Accuracy to predict the author= 66.66666666666666%

Author: Louisa May Alcott
[7, 7, 7]
Accuracy to predict the author= 100.0%

Author: Voltaire
[8, 8, 8]
Accuracy to predict the author= 100.0%

Author: William Shakespeare
[9, 9, 9]
Accuracy to predict the author= 100.0%

Model Accuracy= 80.0%


# Method 3: Using Jaccard similarity

In [18]:
# Predict the author of the book given in input
def method3_jaccard_similarity(list1top100):
    
    list1top100Words = set([tup[0] for tup in list1top100])
    max_jaccard = -1
    author_index = -1
    
    #Check which authors have the greatest number of words in common compared to the book in input
    for a in range(n_authors):
        
        #Take top 100 of training set wordcount
        list2top100 = trainingSetWordCountAuthors[a][1:101]
        list2top100Words = set([tup[0] for tup in list2top100])
    
        # Calculate jaccard similarity
        intersection = len(list1top100Words.intersection(list2top100Words))
        union = len(list1top100Words.union(list2top100Words))
        jaccard_similarity = intersection / union

    
        if jaccard_similarity > max_jaccard:
            author_index = a
            max_jaccard = jaccard_similarity 
            
    return author_index

In [19]:
method = 3
testing(method)


Author: Arthur Conan Doyle
[5, 5, 1]
Accuracy to predict the author= 0.0%

Author: Charles Dickens
[1, 1, 1]
Accuracy to predict the author= 100.0%

Author: Daniel Defoe
[2, 2, 2]
Accuracy to predict the author= 100.0%

Author: Edith Wharton
[3, 0, 3]
Accuracy to predict the author= 66.66666666666666%

Author: Jane Austen
[4, 4, 4]
Accuracy to predict the author= 100.0%

Author: Joseph Conrad
[5, 5, 5]
Accuracy to predict the author= 100.0%

Author: Lewis Carroll
[6, 7, 6]
Accuracy to predict the author= 66.66666666666666%

Author: Louisa May Alcott
[7, 7, 7]
Accuracy to predict the author= 100.0%

Author: Voltaire
[7, 8, 8]
Accuracy to predict the author= 66.66666666666666%

Author: William Shakespeare
[9, 9, 8]
Accuracy to predict the author= 66.66666666666666%

Model Accuracy= 76.66666666666667%


# ML Method

In [31]:
wordsCountAuthorsBooks[0][0][0]
documents = " ".join([f"{word}({frequency})" for word, frequency in wordsCountAuthorsBooks[0][0]])
#print(documents)
print(len(wordsCountAuthorsBooks[0]))

10


In [51]:
import random
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [63]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [356]:
SetWords = set()

for a in range(n_authors):
    for b in range(10):
        count = 1
        for tup in wordsCountAuthorsBooks[a][b]:
            SetWords.add(tup[0])
            if count == 100:
                break

In [357]:
dictSetWords = {}
for index, word in enumerate(SetWords):
    dictSetWords[word] = index

In [313]:
print(dictSetWords)

{'dark—made': 0, 'blunders': 1, '87': 2, 'scent': 3, 'est': 4, 'omitting': 5, 'ungarlanding': 6, 'volcanoes': 7, 'short—oh': 8, 'wimpole': 9, 'purvey': 10, 'lushingtons': 11, 'suggestive': 12, 'communes': 13, 'scapegraces': 14, 'coherently': 15, 'sackree': 16, 'limbes': 17, 'deserved': 18, 'serried': 19, 'generating': 20, '—should': 21, 'observes': 22, "stick's": 23, 'chosen': 24, 'legislation': 25, 'impies': 26, 'degrees—the': 27, "sanch's": 28, 'délices': 29, 'reconstructing': 30, 'nelson': 31, 'boy': 32, 'unfounded': 33, 'barbadoes': 34, 'settee': 35, 'corrupted': 36, 'me—viz': 37, 'importunities': 38, 'côté': 39, 'father—mr': 40, 'hinds': 41, 'franknesse': 42, 'banket': 43, 'fashionables': 44, 'barmaid': 45, 'coinage': 46, 'grammer': 47, 'tantalised': 48, 'make—': 49, 'monotony': 50, 'depopulated': 51, 'shrive': 52, 'misers': 53, 'cheerfulnesses': 54, 'jefferson': 55, "declin'd": 56, 'hounded': 57, 'livingston': 58, 'tourene': 59, 'ignoble': 60, 'pimping': 61, '—watching': 62, 'cro

In [358]:
def relative_frequences(tuple_list):

    result = []

    total_words = tuple_list[-1][1]
    #print(total_tuple) #[0] total_words [1] number
    
    count = 1
    for tup in tuple_list:
        if tup[0] not in punctuations_char and tup[0] not in function_words:
            new_tuple = (tup[0], tup[1]/total_words)
            result.append(new_tuple)
            count += 1
        #if count == 50:
        #    break

    return(result)

In [359]:
documents = []
author_labels = []


for a in range(10):

    tmpDocument = np.zeros(len(dictSetWords))
    for b in range(10):
        #document = " ".join([f"{word}({frequency})" for word, frequency in relative_frequences(wordsCountAuthorsBooks[a][b])])
        
        #for word, frequency in relative_frequences(wordsCountAuthorsBooks[a][b]):
        for word, frequency in wordsCountAuthorsBooks[a][b]:
  
            tmpDocument[dictSetWords[word]] = frequency
         
        documents.append(tmpDocument)
        author_labels.append(a)

#print(documents)
#print(author_labels)

In [360]:
print(list(documents[0]))
#print(author_labels[0])

[0.0, 1.0, 2.0, 6.0, 3.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 2.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 9.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 2.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 1.0, 1.0, 1.0, 22.0, 7.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 2.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 1.0, 0

In [361]:

X = documents

len(X)

100

In [362]:
y = np.array(author_labels)
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8,
       8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9])

In [363]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [364]:
#classifier = SVC(kernel='poly', degree=3)
#classifier = SVC(kernel='linear')
#classifier = SVC(kernel='sigmoid')
classifier = RandomForestClassifier(n_estimators=100, random_state=42)

#Training
classifier.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [365]:
#Prediction
y_pred = classifier.predict(X_test)

In [366]:
y_test

array([8, 5, 7, 4, 4, 3, 2, 8, 1, 0, 1, 3, 7, 3, 9, 0, 7, 7, 1, 3])

In [367]:
y_pred

array([8, 5, 7, 4, 4, 3, 2, 8, 1, 0, 1, 3, 7, 3, 9, 0, 7, 7, 1, 3])

In [368]:
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

1.0


In [ ]:
##########################################

TypeError: 'ellipsis' object is not iterable

# Celle di Test

In [21]:
#To find same words in top200 words of all authors
same_words = []

#take first author to compare the same words of the other authors 
first = trainingSetWordCountAuthors[0]

#Compare words of the first author to others to find same words
for _, word in first:
    count = 0
    for i in range(1,10):
        for _, word2 in trainingSetWordCountAuthors[i]:
            if word == word2:
                count += 1
                continue
    if count == 9:
        same_words.append(word)

print(same_words)
print(len(same_words))
same_words = ['total_words']

['total_words', 'man', 'little', 'see', 'great', 'time', 'day', 'come', 'came', 'own', 'know', 'think', 'good', 'made', 'hand', 'long', 'house', 'saw', 'heard', 'old', 'thought', 'take', 'go', 'make', 'life', 'give', 'put', 'place', 'set', 'thing']
30
